In [1]:
import numpy as np
import pandas as pd
import emoji
from glove_vec import read_glove_vectors

from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,SimpleRNN,LSTM,Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.utils import np_utils

Using TensorFlow backend.
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inderpreet/.local/lib/python3.6/site-packages/t

In [2]:
train=pd.read_csv("./train_emoji.csv",header=None)

test=pd.read_csv("./test_emoji.csv",header=None)

In [3]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [4]:
emoji_dict = {"0" : ":heart:", "1" : ":baseball:", "2" : ":smile:", 
              "3" : ":disappointed:", "4" : ":fork_and_knife:"}

In [5]:
def printdict(label):
    
    return emoji.emojize(emoji_dict[str(label)],use_aliases=True)

In [6]:
for i in range(5):
    
    print(printdict(i))

❤
⚾
😄
😞
🍴


In [7]:
X_train=train[0]
Y_train=train[1]

X_test=test[0]

Y_test=test[1]

print(X_train.shape,Y_train.shape)

(132,) (132,)


In [8]:
for ix in range(X_train.shape[0]):
    X_train[ix]=X_train[ix].split()
    
for ix in range(X_test.shape[0]):
    X_test[ix]=X_test[ix].split()

Y_train=np_utils.to_categorical(Y_train)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
print(X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [10]:
emb_matrix=read_glove_vectors("./glove.6B.50d.txt")

In [11]:
print(emb_matrix["i"],emb_matrix["i"].shape)

[ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-01  8.8180e-01  2.9561e-01
  1.3548e+00 -2.5701e+00 -1.3523e+00  4.5880e-01  1.0068e+00 -1.1856e+00
  3.4737e+00  7.7898e-01 -7.2929e-01  2.5102e-01 -2.6156e-01 -3.4684e-01
  5.5841e-01  7.5098e-01  4.9830e-01 -2.6823e-01 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01] (50,)


In [12]:
embeddings_train=np.zeros((X_train.shape[0],10,50))
embeddings_test=np.zeros((X_test.shape[0],10,50))


In [13]:

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embeddings_train[ix][ij] = emb_matrix[X_train[ix][ij].lower()]
        

for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embeddings_test[ix][ij] = emb_matrix[X_test[ix][ij].lower()]

In [14]:
print(embeddings_train.shape,embeddings_test.shape)

(132, 10, 50) (56, 10, 50)


In [15]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.summary()


W1122 21:56:44.134474 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1122 21:56:44.173711 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1122 21:56:44.176674 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1122 21:56:44.360600 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.place

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                5184      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 34,709
Trainable params: 34,709
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

W1122 21:56:44.523333 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1122 21:56:44.527972 140492492531520 deprecation_wrapper.py:119] From /home/inderpreet/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [17]:
hist=model.fit(embeddings_train,Y_train,epochs=50,batch_size=12,validation_split=0.20,shuffle=True)

W1122 21:56:44.748145 140492492531520 deprecation.py:323] From /home/inderpreet/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 105 samples, validate on 27 samples
Epoch 1/50
105/105 [==============================] - 2s 18ms/step - loss: 1.5751 - acc: 0.3048 - val_loss: 1.6329 - val_acc: 0.2222
Epoch 2/50
105/105 [==============================] - 0s 2ms/step - loss: 1.5403 - acc: 0.2952 - val_loss: 1.6202 - val_acc: 0.2222
Epoch 3/50
105/105 [==============================] - 0s 2ms/step - loss: 1.4901 - acc: 0.3524 - val_loss: 1.5990 - val_acc: 0.2222
Epoch 4/50
105/105 [==============================] - 0s 2ms/step - loss: 1.4249 - acc: 0.4190 - val_loss: 1.5498 - val_acc: 0.2593
Epoch 5/50
105/105 [==============================] - 0s 2ms/step - loss: 1.3686 - acc: 0.5619 - val_loss: 1.5168 - val_acc: 0.2963
Epoch 6/50
105/105 [==============================] - 0s 2ms/step - loss: 1.2848 - acc: 0.5429 - val_loss: 1.3839 - val_acc: 0.3704
Epoch 7/50
105/105 [==============================] - 0s 2ms/step - loss: 1.2456 - acc: 0.4476 - val_loss: 1.2832 - val_acc: 0.3704
Epoch 8/50
105/105 [==========

In [18]:
pred=model.predict_classes(embeddings_test)

In [19]:
acc=float(sum(pred==Y_test)/embeddings_test.shape[0])

In [28]:
print(acc)

0.625


In [31]:
def print_emoji(label):
    
    return emoji.emojize(emoji_dict[label], use_aliases=True)

In [32]:
for ix in range(embeddings_test.shape[0]):
    
    if (pred[ix] != Y_test[ix]):
        print(X_test[ix], print_emoji(Y_test[ix]), print_emoji(pred[ix]))

KeyError: 0